# Example of code for custom optimisation

This notebook shows how to configure and run a customised configuration for the solver. To be used the `asprin` optimiser tool must be installed, and the easiest way to do it is via its [Anaconda package](https://anaconda.org/potassco/asprin).

In [ ]:
from pathlib import Path

import negdis

Using `cerv` dataset as an example:

In [ ]:
DATA_DIR = Path('data', 'synt')

POS_LOG_PATH = DATA_DIR.joinpath('declarative_traces_positives.xml')
NEG_LOG_PATH = DATA_DIR.joinpath('declarative_traces_negatives1.xml')

DECLARE_RULES_PATH = DATA_DIR.joinpath('declare_rules.txt')

Input files can also be created on the fly, in this case the set of Declare patterns are defined directly in this notebook as a string. See below on how to use it (note the use of `r''` Python construct to avoid string interpolation). Strings can be passed as file arguments by using the `negdis.as_file` context function (see below for its usage).

In [ ]:
DECLARE_PATTERNS_STR = r'''
Absence(a):[^a]*
Absence2(a):[^a]*(a)?[^a]*
Absence3(a):[^a]*((a)?[^a]*){2}
AlternatePrecedence(a,b):[^b]*(a[^b]*b[^b]*)*[^b]*
Condition(a,b):[^b]*(a.*b)*[^b]*
Existence(a):.*a.*
NotSuccession(a,b):[^a]*(a[^b]*)*[^ab]*
Response(a,b):[^a]*(a.*b)*[^a]*
'''

## Choices generation stage

Run `negdis` to generate the candidates, output is stored in a temporary file. *Negdis* executable is wrapped in the `negdis.Negdis` class; its `default` method returns an object using the file in the `dist` directory.

In [ ]:
negdis_exe = negdis.Negdis.default()
print(negdis_exe.version())

In [ ]:
import atexit
import tempfile

out_dir = Path(tempfile.mkdtemp())

@atexit.register
def cleanup():
    import shutil
    # remove the temporary directory on termination
    shutil.rmtree(out_dir)


# output on a temporary file
CHOICES_PATH = Path(out_dir).joinpath('choices.json')

with negdis.as_file(DECLARE_PATTERNS_STR) as patterns:
    negdis_exe.discover(POS_LOG_PATH, NEG_LOG_PATH, patterns, CHOICES_PATH)

print(f'Choices written to: {CHOICES_PATH}')

### Shows top constraints

In [ ]:
negdis.count_choices(CHOICES_PATH)

## Optimisation stage

The configuration of the solver can be customised using the class `negdis.SolverConf`:

In [ ]:
opt_code = r'''
%%%%%%%%%%%%%%
%%%%%%% asprin optimisation statements

% Minimal cardinality wrt the closure constraints, and selected constraints for ties

#preference(p1,less(cardinality)){ ${predicate_holds}(C) : ${predicate_constraint}(C) }.
#preference(p2,less(cardinality)){ ${predicate_selected}(C) : ${predicate_choice}(_, C) }.

#preference(p10,lexico){ 1::**p2; 2::**p1 }.
#optimize(p10).
'''

opt_mode = negdis.SolverConf.from_dict({
    'id': 'asprinminclos',
    'inputs': ['guess.lp'],
    'args': ['--quiet=1'],
    'docstring': 'Minimal cardinality wrt the closure constraints, and selected constraints for ties',
    'solver': 'asprin',
    'template': opt_code
})

Actual ASP program is generated by concatenating the content of files in `inputs` with the string `template`, and then replacing the values of `${...}` macros. Default macros are:

- `${predicate_action}(action)`: predicate with all the actions
- `${predicate_choice}(trace, constraint)`: the candidates for each trace
- `${predicate_constraint_action}(constraint, action)`: actions which are argument of the constraint
- `${predicate_constraint_name}(constraint, name, arity)`: name of the pattern of the constraint
- `${predicate_constraint}(constraint)`: all constraints that can be generated by the rules, and the candidates
- `${predicate_holds}(constraint)`: constraints deduced by the rules starting from the selected
- `${predicate_selected}(constraint)`: constraints selected for a specific model

each `constraint` is encoded as the function term `${functor_declare}(pattern, actions+)`. All these predicates except `${predicate_selected}` and `${predicate_holds}` are completely evaluated during grounding, so they can be used as *domain predicates*. 

New macros can be defined (or the default value replaced) by means of a dictionary that can be passed as an argument.

If the file names in `inputs` are not absolute paths, they'll be searched in the current directory followed by the directory of the Python package `negdis.templates`; e.g. the `guess.lp` is fetched from `negdis/templates/guess.lp` within one of the directories in `sys.path`. The location of the `negdis.templates` module directory can be verified using the following code:



In [ ]:
import inspect, os
try:
    print(os.path.dirname(inspect.getfile(negdis.templates)))
except Exception as e:
    print(e)


The ASP program generated by a configuration (without the parts due to the specific problem) can be inspected using the `SolverConf.program` method, which accepts an *optional* argument with the mapping for the macros:

In [ ]:
print(opt_mode.program(mapping={'predicate_holds': 'closure'}))

To show the original ASP program without the variable substitution the `SolverConf.program` method takes an optional `eval` argument which can be set to `False`:

In [ ]:
print(opt_mode.program(eval=False))

All default solver configurations are available via the `negdis.configurations` function:

In [ ]:
negdis.configurations()

The `negdis.optimise_choices` function can be used to run the optimisation code (see below for examples). The function takes an optional `mapping` argument which is used to expand the program.

Note that *asprin* solver doesn't support JSON output, so for the time being only the raw solver output is printed and no statistics are reported.

In [ ]:
negdis.optimise_choices(CHOICES_PATH, opt_mode, DECLARE_RULES_PATH, models=10, timeout=60, mapping=dict())

In [ ]:
negdis.optimise_choices(CHOICES_PATH, 'minclos', DECLARE_RULES_PATH, models=10, timeout=60, mapping=dict())

For debugging purposes, the ASP program can be generated using the `negdis.asp_program` function:

In [ ]:
print(negdis.asp_program(CHOICES_PATH, opt_mode, DECLARE_RULES_PATH, mapping=dict()))

The following example shows how to optimise by preferring models that do not include a specific pattern.

In [ ]:
opt_code = r'''
%%%%%%%%%%%%%%
%%%%%%% asprin optimisation statements

in_model(P) :- ${predicate_selected}(C), ${predicate_constraint_name}(C, P, _).

not_nice_model :- in_model(${bad_constraint}).
nice_model :- not not_nice_model. 

#preference(p1,aso){ nice_model >> not_nice_model }.
#preference(p2,subset){ ${predicate_holds}(C) : ${predicate_constraint}(C) }.

#preference(p10,lexico){ 1::**p2; 2::**p1 }.
#optimize(p10).

#show in_model/1.
#show nice_model/0.
#show not_nice_model/0.
'''

opt_mode_nice = negdis.SolverConf.from_dict({
    'id': 'nice_models',
    'inputs': ['guess.lp'],
    'args': ['--quiet=1'],
    'docstring': 'Models without a specific constraint, and (subset) closure for ties',
    'solver': 'asprin',
    'template': opt_code
})

In the following two cells below preferred models are those without *condition* and *response* respectively.

In [ ]:
negdis.optimise_choices(CHOICES_PATH, opt_mode_nice, DECLARE_RULES_PATH, models=5, timeout=60, mapping={'bad_constraint': 'condition'})

In [ ]:
negdis.optimise_choices(CHOICES_PATH, opt_mode_nice, DECLARE_RULES_PATH, models=5, timeout=60, mapping={'bad_constraint': 'response'})

In [ ]:
import shutil

shutil.rmtree(str(out_dir))

## Command line

If the package is installed via `pip` (or within a *conda* environment) there are two command line scripts available:

- `negdis`: run the choice generation code
- `aspdeclare`: access to the optimisation code

```bash
$ aspdeclare 
usage: aspdeclare.py [-h] [-v] [--version] {version,rules,choices,asp,run} ...

Use clingo ASP solver to minimise the set of constraints that exclude a set of traces. See the negdis project for details.

positional arguments:
  {version,rules,choices,asp,run}
    version
    rules               Convert a set of rules of the form head <- body to a
                        set of ASP rules
    choices             Convert a negdis choice file to a set of ASP facts
    asp                 Generate the ASP file for the input
    run

optional arguments:
  -h, --help            show this help message and exit
  -v, --verbose         increase output verbosity
  --version             show program's version number and exit
```

The `aspdeclare asp` command can be used to generate the ASP program which can be fed directly into the solver. The Solver configuration can be stored in a JSON file and passed via the `--conf` argument.